In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from db_helpers import *
from web3 import Web3
import os, json
from supabase import create_client, Client
import math

# Nightly Script

In [3]:
all_data = {
    "categories": pd.DataFrame(get_data("categories", "*")),
    "comments": pd.DataFrame(get_data("comments", "*")),
    "follows": pd.DataFrame(get_data("follows", "*")),
    "posts": pd.DataFrame(get_data("posts", "*")),
    "users": pd.DataFrame(get_data("users", "*")),
    "votes": pd.DataFrame(get_data("votes", "*")),
}

In [4]:
all_data['users'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6056 entries, 0 to 6055
Data columns (total 22 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       6056 non-null   int64  
 1   created_at               6056 non-null   object 
 2   username                 28 non-null     object 
 3   profile_pic              6056 non-null   object 
 4   msa_id                   0 non-null      object 
 5   transaction_hash         0 non-null      object 
 6   wallet_address_personal  3 non-null      object 
 7   wallet_address_provided  0 non-null      object 
 8   exp                      6056 non-null   int64  
 9   level                    6056 non-null   int64  
 10  reddit_airdrop_value     714 non-null    float64
 11  reddit_airdrop_claimed   6056 non-null   object 
 12  reddit_username          6051 non-null   object 
 13  github_username          0 non-null      object 
 14  discord_username        

In [7]:
# Daily payout
for idx, row in all_data['users'][all_data['users']['daily_payout_claimed'] == True].iterrows():
    updates = {
        "daily_payout_claimed": False, 
        "tokens_to_claim": row['tokens_to_claim'] + row['daily_payout_value']
    }
    supabase.table("users").update(updates).eq("id", row['id']).execute()
    
all_users = all_data['users'].copy()
all_users.loc[all_users['username'].isna(), :] = update_scores(all_data, all_users[all_users['username'].isna()])
all_users.loc[~all_users['username'].isna(), :] = update_scores(all_data, all_users[~all_users['username'].isna()])

for idx, row in all_users.iterrows():
    updates = {
        'social_score': row['social_score'],
        'level': row['level'],
        'exp': row['exp'],
        'exp_to_next_level': row['exp_to_next_level'],
        'daily_payout_value': row['daily_payout_value'],
    }
    supabase.table("users").update(updates).eq("id", row['id']).execute()

F:\web3_messing_around\PostThread-Polygon\backend\db_helpers.py:142: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['social_score'] = social_score
F:\web3_messing_around\PostThread-Polygon\backend\db_helpers.py:143: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['level'] = level
F:\web3_messing_around\PostThread-Polygon\backend\db_helpers.py:144: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the ca

In [52]:
# Reddit Airdrop

# Update airdrop value
for idx, row in all_data['users'][~all_data['users']['reddit_username'].isna() & all_data['users']['reddit_airdrop_value'].isna()  & ~all_data['users']['username'].isna()].iterrows():
    user = reddit.redditor(row['reddit_username'])   
    try:
        airdrop_value = user.total_karma
    except:
        continue
        
    supabase.table("users").update({"reddit_airdrop_value": airdrop_value}).eq("id", row['id']).execute()
print("Done update airdrop value")
                
# Check for claims
for idx, row in all_data['users'][all_data['users']['reddit_airdrop_claimed'] == "pending"].iterrows():
    if row['reddit_id'] is None:
        supabase.table("users").update({"reddit_airdrop_claimed": "rejected"}).eq("id", row['id']).execute()
    
    user = reddit.redditor(reddit_username)  
    try:
        for i, post in enumerate(user.new()):
            if i > 10:
                break
            if type(post) == praw.models.reddit.submission.Submission:
                if row["wallet_address_personal"] in post.selftext:
                    airdrop_value = user.total_karma
                    updates = {
                        "reddit_airdrop_value": airdrop_value,
                        "reddit_airdrop_claimed": "claimed",
                        "tokens_to_claim": row['tokens_to_claim'] + airdrop_value,
                    }
                    supabase.table("users").update(updates).eq("id", row['id']).execute()
    except:
        supabase.table("users").update({"reddit_airdrop_claimed": "rejected"}).eq("id", row['id']).execute()

Done update airdrop value


In [8]:
# Transfer tokens

for idx, row in all_data['users'][(all_data['users']['tokens_claimed'] == True) & (all_data['users']['tokens_to_claim'] != 0) & (~all_data['users']['wallet_address_personal'].isna())].iterrows():
    with open("new_announcements.txt", "a") as file_object:
        file_object.write(json.dumps({"type": "mint_tokens", "wallet": row["wallet_address_personal"], "amount": row["tokens_to_claim"]}) + '\n')
    supabase.table("users").update({"tokens_claimed": False, "tokens_to_claim": 0}).eq("id", row['id']).execute()

In [ ]:
# Mint users
top_users = all_data['users'][!all_data['users']['username'].isna() & all_data['users']['msa_id'].isna()]
top_users = top_users.sort_values("exp")

for idx, row in top_users.iterrows():
    wallet = get_wallet_from_username(row['username'])
    with open("new_announcements.txt", "a") as file_object:
        file_object.write(json.dumps({"type": "new_user", "private_key": wallet.privateKey}) + '\n')
    supabase.table("users").update({"wallet_address_provided": wallet.address).eq("id", row['id']).execute()
    
# Mint users with msa
top_users = all_data['users'][!all_data['users']['username'].isna() & all_data['users']['transaction_hash'].isna() & ~all_data['users']['msa_id'].isna()]
top_users = top_users.sort_values("exp")

for idx, row in top_users.iterrows():
    wallet = get_wallet_from_username(row['username'])
    message = mint_user(row['msa_id'], row['profile_pic'], wallet.address)
    with open("new_announcements.txt", "a") as file_object:
        file_object.write(json.dumps({"type": "mint_data", "message": message, "amount": row["msa_id"]}) + '\n')
    supabase.table("users").update({"tokens_claimed": False, "tokens_to_claim": 0}).eq("id", row['id']).execute()

In [ ]:
# Mint posts
top_posts = all_data['posts'][all_data['posts']['reddit_id'].isna() & all_data['posts']['transaction_hash'].isna() & ~all_data['posts']['ipfs_hash'].isna()]
top_posts = top_posts.join(all_data['users'].set_index('id')[['username', 'msa_id', 'tokens_to_claim']], on="user_id")
top_posts = top_posts[~top_posts['msa_id'].isna() & ~top_posts['username'].isna() & top_posts['tokens_to_claim'] > 5]
top_posts = top_posts.join(all_data['votes'].groupby("post_id").sum()[['up']], on="id", how="inner")
top_posts = top_posts.sort_values("up")

for idx, row in top_posts.iterrows():
    
    
    with open("new_announcements.txt", "a") as file_object:
        file_object.write(json.dumps({"type": "mint_tokens", "wallet": row["wallet_address_personal"], "amount": row["tokens_to_claim"]}) + '\n')
    
    supabase.table("users").update({"tokens_claimed": False, "tokens_to_claim": 0}).eq("id", row['id']).execute()